# <font color="red">Solar Energy Prediction with Support Vector Regressor</font>

# Library Import

In [1]:
# Data manipulation and visualization
import pandas as pd
import math
import json

# Preprocessing
from sklearn.model_selection import StratifiedShuffleSplit, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.svm import SVR

import sys
# sys.path.append('../../../') # Uncomment this line if running locally
sys.path.append('/kaggle/input/weatherdata') # Uncomment this line if running on Kaggle
from historyManagement import *

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Datasets Import

In [2]:
# Load the weather dataset for visibility prediction
# weather_data = pd.read_csv('../../../FinalDatasets/finalDataset.csv') # Uncomment this line if running locally
weather_data = pd.read_csv('/kaggle/input/weatherdata/finalDataset.csv') # Uncomment this line if running on Kaggle
weather_data['DateTime'] = pd.to_datetime(weather_data['DateTime'])
weather_data.head()

,DateTime,Year,Month,Day,Hour,DaySegments,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,...,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy,Conditions,Icon
0,2023-01-01 00:00:00,2023,1,1,0,Late Night,0,0,0,1,...,0.49,1018.68,0.03,0.0,10.0,2.76,0.0,0.0,Clear,clear-night
1,2023-01-01 01:00:00,2023,1,1,1,Late Night,0,0,0,1,...,0.54,1018.03,0.11,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
2,2023-01-01 02:00:00,2023,1,1,2,Late Night,0,0,0,1,...,30.51,1017.56,0.03,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
3,2023-01-01 03:00:00,2023,1,1,3,Late Night,0,0,0,1,...,49.23,1018.05,0.00,0.0,10.0,2.28,0.0,0.0,Clear,clear-night
4,2023-01-01 04:00:00,2023,1,1,4,Late Night,0,0,0,1,...,49.90,1018.00,86.17,0.0,10.0,1.27,0.0,0.0,Partially cloudy,fog


In [3]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

In [4]:
weather_data.describe()

,DateTime,Year,Month,Day,Hour,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,DaySegments_Midday,...,Windgust,WindSpeed,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy
count,14616,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,...,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000
mean,2023-11-01 11:30:00,2023.400657,5.720854,15.735632,11.500000,0.131294,0.060550,0.098043,0.249726,0.116585,...,17.068560,8.090783,159.739178,1008.265012,53.579923,2.263555,16.504787,4.368134,227.640683,0.819217
min,2023-01-01 00:00:00,2023.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.660000,0.000000,0.000000,982.420000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,2023-06-02 05:45:00,2023.000000,3.000000,8.000000,5.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.170000,2.870000,80.965000,1003.277500,26.670000,0.000000,10.000000,3.990000,0.000000,0.000000
50%,2023-11-01 11:30:00,2023.000000,6.000000,16.000000,11.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,14.645000,7.670000,163.790000,1007.970000,51.540000,0.000000,10.000000,4.650000,10.665000,0.010000
75%,2024-04-01 17:15:00,2024.000000,8.000000,23.000000,17.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.410000,11.750000,241.402500,1013.400000,87.790000,4.740000,12.580000,4.650000,459.745000,1.660000
max,2024-08-31 23:00:00,2024.000000,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,84.120000,71.970000,360.000000,1022.060000,100.000000,10.000000,97.740000,24.100000,1026.650000,3.700000
std,NaN,0.490048,3.195073,8.803610,6.922423,0.337734,0.238511,0.297383,0.432869,0.320936,...,10.195911,6.250203,104.422027,6.010683,35.166637,3.083074,15.031827,1.376994,307.219387,1.106444


# Data Split

In [5]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for train_idx, test_idx in sss.split(weather_data, weather_data['DaySegments']):
    weather_data_train = weather_data.iloc[train_idx]
    weather_data_test = weather_data.iloc[test_idx]

In [6]:
# Show the count of each unique class in DaySegments
class_counts = weather_data['DaySegments'].value_counts().reset_index()
class_counts.columns = ['DaySegments', 'Count']

print("Distribution of data based on DaySegments:")
print(class_counts)

Distribution of data based on DaySegments:
     DaySegments  Count
0     Late Night   3650
1        Morning   3504
2      Afternoon   1919
3         Midday   1704
4          Night   1521
5        Evening   1433
6  Early Morning    885


In [7]:
print("Distribution of DaySegments in Training Set:")
print(weather_data_train['DaySegments'].value_counts())

print("\nDistribution of DaySegments in Test Set:")
print(weather_data_test['DaySegments'].value_counts())

Distribution of DaySegments in Training Set:
DaySegments
Late Night       2920
Morning          2803
Afternoon        1535
Midday           1363
Night            1217
Evening          1146
Early Morning     708
Name: count, dtype: int64

Distribution of DaySegments in Test Set:
DaySegments
Late Night       730
Morning          701
Afternoon        384
Midday           341
Night            304
Evening          287
Early Morning    177
Name: count, dtype: int64


# Some Reused Parameters

In [8]:
grid_param = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'epsilon': [0.1, 0.2],
    'coef0': [0.0, 0.1, 1.0],
    'shrinking': [True],
    'tol': [1e-3, 1e-4]
}

In [9]:
number_of_model_for_randomized_grid = 130
model_name_for_saving = "SVR"
target_name_for_saving = "SolarEnergy"

# All Features

In [10]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_train = weather_data_train['SolarEnergy']

In [11]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_test = weather_data_test['SolarEnergy']

In [12]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [13]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


## MinMax Scaler

In [14]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [15]:
X_train

array([[0.72727273, 0.83333333, 0.43478261, ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.26666667, 0.69565217, ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.43333333, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.86666667, 0.43478261, ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.4       , 0.91304348, ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.26666667, 0.34782609, ..., 0.8925    , 0.29      ,
        0.27812962]])

In [16]:
X_test

array([[0.63636364, 0.06666667, 0.95652174, ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.26666667, 0.65217391, ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.03333333, 0.2173913 , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.2       , 0.73913043, ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.13333333, 0.86956522, ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.73333333, 0.95652174, ..., 0.5161    , 0.        ,
        0.07388643]])

### Default Parameters

In [17]:
training_model = SVR()
training_model

SVR()

In [18]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 1.8 s, sys: 96.1 ms, total: 1.89 s
Wall time: 1.89 s


SVR()

In [19]:
y_pred = training_model.predict(X_test)
y_pred

array([ 0.02183556,  0.28229787,  0.0646771 , ...,  0.765373  ,
        0.06056244, -0.02290922])

In [20]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [21]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [22]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.005349976789730829
MAE Score: 0.057441035556980866
R2 Score: 0.9956090024377426
RMSE Score: 0.0731435355293332


In [23]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.005350, R²: 0.995609
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [24]:
grid_search = RandomizedSearchCV(
    SVR(), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [25]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 130 candidates, totalling 650 fits
CPU times: user 7.64 s, sys: 1.65 s, total: 9.29 s
Wall time: 15min 46s


RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=130, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100],
                                        'coef0': [0.0, 0.1, 1.0],
                                        'epsilon': [0.1, 0.2],
                                        'gamma': ['scale', 'auto', 0.001, 0.01,
                                                  0.1],
                                        'kernel': ['linear', 'rbf', 'sigmoid'],
                                        'shrinking': [True],
                                        'tol': [0.001, 0.0001]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [26]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

# Save the best parameters as an array
grid_param_minmax = {k: [v] for k, v in grid_search.best_params_.items()}

TUNING RESULTS
Best Parameters: {'tol': 0.001, 'shrinking': True, 'kernel': 'rbf', 'gamma': 0.1, 'epsilon': 0.1, 'coef0': 0.1, 'C': 10}
Best Cross-Validation Score (Negative MSE): -0.0047868140187188515
Best Cross-Validation Score (MSE): 0.0047868140187188515
Best Cross-Validation Score (RMSE): 0.06918680523567229


In [27]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [28]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [29]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.0048228168608383856
MAE Score: 0.05404502028701287
R2 Score: 0.9960416693545655
RMSE Score: 0.06944650358973002


In [30]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004823, R²: 0.996042
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Standard Scaler

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [32]:
X_train

array([[ 1.01884116,  1.17229173, -0.21938027, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461, -0.75947244,  0.64624572, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -0.1913065 , -1.66209024, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943,  1.28592492, -0.21938027, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264, -0.30493969,  1.36760071, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.75947244, -0.50792226, ...,  1.01722469,
         0.20430944,  0.85928021]])

In [33]:
X_test

array([[ 0.70637943, -1.44127155,  1.51187171, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598, -0.75947244,  0.50197472, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -1.55490474, -0.94073525, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092, -0.98673881,  0.79051672, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747, -1.21400518,  1.22332971, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  0.83139217,  1.51187171, ..., -0.05293696,
        -0.73592134, -0.43343649]])

### Default Parameters

In [34]:
training_model = SVR()
training_model

SVR()

In [35]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 2.49 s, sys: 49.7 ms, total: 2.54 s
Wall time: 2.54 s


SVR()

In [36]:
y_pred = training_model.predict(X_test)
y_pred

array([-0.0229861 ,  0.39469724,  0.06991245, ...,  0.76831368,
        0.12537097, -0.03752152])

In [37]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [38]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [39]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.006366638814458079
MAE Score: 0.06155000710403795
R2 Score: 0.9947745762995236
RMSE Score: 0.07979122015897538


In [40]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.006367, R²: 0.994775
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [41]:
grid_search = RandomizedSearchCV(
    SVR(), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [42]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 130 candidates, totalling 650 fits
CPU times: user 10min 52s, sys: 5.76 s, total: 10min 57s
Wall time: 1h 5min 33s


RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=130, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100],
                                        'coef0': [0.0, 0.1, 1.0],
                                        'epsilon': [0.1, 0.2],
                                        'gamma': ['scale', 'auto', 0.001, 0.01,
                                                  0.1],
                                        'kernel': ['linear', 'rbf', 'sigmoid'],
                                        'shrinking': [True],
                                        'tol': [0.001, 0.0001]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [43]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

# Use the best parameters to train the robust scaler model
grid_param_standard = {k: [v] for k, v in grid_search.best_params_.items()}

TUNING RESULTS
Best Parameters: {'tol': 0.001, 'shrinking': True, 'kernel': 'linear', 'gamma': 0.001, 'epsilon': 0.1, 'coef0': 1.0, 'C': 100}
Best Cross-Validation Score (Negative MSE): -0.004823418439956266
Best Cross-Validation Score (MSE): 0.004823418439956266
Best Cross-Validation Score (RMSE): 0.06945083469589308


In [44]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [45]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [46]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004812194287321549
MAE Score: 0.051973282910879715
R2 Score: 0.9960503878399442
RMSE Score: 0.06936998116852526


In [47]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004812, R²: 0.996050
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Robust Scaler

In [48]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [49]:
X_train

array([[ 0.6       ,  0.66666667, -0.16666667, ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        , -0.46666667,  0.33333333, ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -0.13333333, -1.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       ,  0.73333333, -0.16666667, ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        , -0.2       ,  0.75      , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.46666667, -0.33333333, ...,  0.61755563,
         0.59917355,  7.5       ]])

In [50]:
X_test

array([[ 4.00000000e-01, -8.66666667e-01,  8.33333333e-01, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -4.66666667e-01,  2.50000000e-01, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -9.33333333e-01, -5.83333333e-01, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01, -6.00000000e-01,  4.16666667e-01, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01, -7.33333333e-01,  6.66666667e-01, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  4.66666667e-01,  8.33333333e-01, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]])

### Default Parameters

In [51]:
training_model = SVR()
training_model

SVR()

In [52]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 6.08 s, sys: 0 ns, total: 6.08 s
Wall time: 6.07 s


SVR()

In [53]:
y_pred = training_model.predict(X_test)
y_pred

array([ 0.03909888,  0.35062476, -0.03044617, ...,  0.97852622,
        0.16758405,  0.06355369])

In [54]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [55]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [56]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.1311540190190274
MAE Score: 0.2522152192049509
R2 Score: 0.89235523808286
RMSE Score: 0.3621519280896174


In [57]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.131154, R²: 0.892355
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [58]:
grid_param_robust = [grid_param_minmax, grid_param_standard]

In [59]:
grid_search = GridSearchCV(
    SVR(), 
    grid_param_robust, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [60]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 2 candidates, totalling 10 fits
CPU times: user 1h 40min 24s, sys: 10.7 s, total: 1h 40min 35s
Wall time: 4h 1min 33s


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid=[{'C': [10], 'coef0': [0.1], 'epsilon': [0.1],
                          'gamma': [0.1], 'kernel': ['rbf'],
                          'shrinking': [True], 'tol': [0.001]},
                         {'C': [100], 'coef0': [1.0], 'epsilon': [0.1],
                          'gamma': [0.001], 'kernel': ['linear'],
                          'shrinking': [True], 'tol': [0.001]}],
             scoring='neg_mean_squared_error', verbose=1)

In [61]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'C': 100, 'coef0': 1.0, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'linear', 'shrinking': True, 'tol': 0.001}
Best Cross-Validation Score (Negative MSE): -0.04859389296050123
Best Cross-Validation Score (MSE): 0.04859389296050123
Best Cross-Validation Score (RMSE): 0.2204402253684686


In [62]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [63]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [64]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.005476284510369002
MAE Score: 0.05747818957857631
R2 Score: 0.9955053352789466
RMSE Score: 0.0740019223423892


In [65]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.005476, R²: 0.995505
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


# Selected Features

In [66]:
numeric_cols = weather_data.select_dtypes(include=[float, int]).columns
correlations = weather_data[numeric_cols].corr()['SolarEnergy'].sort_values(ascending=False)
print(correlations)

SolarEnergy                  1.000000
SolarRadiation               0.999867
UVIndex                      0.998178
DaySegments_Midday           0.594829
Temp                         0.400703
FeelsLike                    0.344667
DaySegments_Morning          0.243917
WindSpeed                    0.216874
DaySegments_Afternoon        0.216045
WindDir                      0.181173
SevereRisk                   0.125741
Hour                         0.095805
Season_Summer                0.093686
Visibility                   0.054855
SeaLevelPressure             0.026544
Year                         0.022062
Dew                          0.014013
Day                         -0.001489
PrecipProb                  -0.013347
Season_Rainy                -0.032901
Season_Autumn               -0.035246
Season_Winter               -0.035617
Precip                      -0.039120
CloudCover                  -0.055435
Month                       -0.061170
Windgust                    -0.100191
DaySegments_

In [67]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

## Run Again

In [68]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Energy
                                                'Dew', # low correlation with Solar Energy
                                                'Precip', # low correlation with Solar Energy
                                                'PrecipProb']) # low correlation with Solar Energy
y_train = weather_data_train['SolarEnergy']

In [69]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Energy
                                                'Dew', # low correlation with Solar Energy
                                                'Precip', # low correlation with Solar Energy
                                                'PrecipProb']) # low correlation with Solar Energy
y_test = weather_data_test['SolarEnergy']

In [70]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [71]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


### MinMax Scaler

In [72]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [73]:
X_train

array([[0.72727273, 0.43478261, 0.        , ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.69565217, 1.        , ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.        , 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.43478261, 0.        , ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.91304348, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.34782609, 0.        , ..., 0.8925    , 0.29      ,
        0.27812962]])

In [74]:
X_test

array([[0.63636364, 0.95652174, 0.        , ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.65217391, 1.        , ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.2173913 , 0.        , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.73913043, 1.        , ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.86956522, 0.        , ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.95652174, 0.        , ..., 0.5161    , 0.        ,
        0.07388643]])

#### Default Parameters

In [75]:
training_model = SVR()
training_model

SVR()

In [76]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 1.49 s, sys: 69.3 ms, total: 1.56 s
Wall time: 1.56 s


SVR()

In [77]:
y_pred = training_model.predict(X_test)
y_pred

array([ 0.02273776,  0.32543406,  0.06658607, ...,  0.73801548,
        0.07418813, -0.00725159])

In [78]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [79]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [80]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.005303970190686729
MAE Score: 0.05780070541468964
R2 Score: 0.9956467623892696
RMSE Score: 0.07282836116985422


In [81]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 0.005350 → 0.005304 (lower is better)
   R²:  0.995609 → 0.995647 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [82]:
grid_search = RandomizedSearchCV(
    SVR(), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [83]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 130 candidates, totalling 650 fits
CPU times: user 11.8 s, sys: 1.22 s, total: 13 s
Wall time: 13min 52s


RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=130, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100],
                                        'coef0': [0.0, 0.1, 1.0],
                                        'epsilon': [0.1, 0.2],
                                        'gamma': ['scale', 'auto', 0.001, 0.01,
                                                  0.1],
                                        'kernel': ['linear', 'rbf', 'sigmoid'],
                                        'shrinking': [True],
                                        'tol': [0.001, 0.0001]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [84]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

# Use the best parameters to train the robust scaler model
grid_param_minmax = {k: [v] for k, v in grid_search.best_params_.items()}

TUNING RESULTS
Best Parameters: {'tol': 0.001, 'shrinking': True, 'kernel': 'rbf', 'gamma': 0.1, 'epsilon': 0.1, 'coef0': 1.0, 'C': 100}
Best Cross-Validation Score (Negative MSE): -0.0046629063873142845
Best Cross-Validation Score (MSE): 0.0046629063873142845
Best Cross-Validation Score (RMSE): 0.0682854771332403


In [85]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [86]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [87]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004739766800727418
MAE Score: 0.05371113641283723
R2 Score: 0.9961098327552353
RMSE Score: 0.0688459643024006


In [88]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 0.004823 → 0.004740 (lower is better)
   R²:  0.996042 → 0.996110 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


### Standard Scaler

In [89]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [90]:
X_train

array([[ 1.01884116, -0.21938027, -0.38875095, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461,  0.64624572,  2.57234098, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -1.66209024, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943, -0.21938027, -0.38875095, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264,  1.36760071, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.50792226, -0.38875095, ...,  1.01722469,
         0.20430944,  0.85928021]])

In [91]:
X_test

array([[ 0.70637943,  1.51187171, -0.38875095, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598,  0.50197472,  2.57234098, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -0.94073525, -0.38875095, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092,  0.79051672,  2.57234098, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747,  1.22332971, -0.38875095, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  1.51187171, -0.38875095, ..., -0.05293696,
        -0.73592134, -0.43343649]])

#### Default Parameters

In [92]:
training_model = SVR()
training_model

SVR()

In [93]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 2.05 s, sys: 94.2 ms, total: 2.15 s
Wall time: 2.15 s


SVR()

In [94]:
y_pred = training_model.predict(X_test)
y_pred

array([-0.03589382,  0.37586495,  0.05148853, ...,  0.75950185,
        0.14547303, -0.05208922])

In [95]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [96]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [97]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.005930239911598078
MAE Score: 0.060641094750918424
R2 Score: 0.9951327510344697
RMSE Score: 0.07700805095311838


In [98]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 0.006367 → 0.005930 (lower is better)
   R²:  0.994775 → 0.995133 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [99]:
grid_search = RandomizedSearchCV(
    SVR(), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [100]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 130 candidates, totalling 650 fits
CPU times: user 7min 10s, sys: 3.22 s, total: 7min 13s
Wall time: 47min 11s


RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=130, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100],
                                        'coef0': [0.0, 0.1, 1.0],
                                        'epsilon': [0.1, 0.2],
                                        'gamma': ['scale', 'auto', 0.001, 0.01,
                                                  0.1],
                                        'kernel': ['linear', 'rbf', 'sigmoid'],
                                        'shrinking': [True],
                                        'tol': [0.001, 0.0001]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [101]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

# Use the best parameters to train the robust scaler model
grid_param_standard = {k: [v] for k, v in grid_search.best_params_.items()}

TUNING RESULTS
Best Parameters: {'tol': 0.001, 'shrinking': True, 'kernel': 'linear', 'gamma': 0.001, 'epsilon': 0.1, 'coef0': 1.0, 'C': 100}
Best Cross-Validation Score (Negative MSE): -0.004765955822107713
Best Cross-Validation Score (MSE): 0.004765955822107713
Best Cross-Validation Score (RMSE): 0.06903590241394483


In [102]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [103]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [104]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004801088429733647
MAE Score: 0.051976584029663334
R2 Score: 0.9960595029810956
RMSE Score: 0.06928988692250584


In [105]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 0.004812 → 0.004801 (lower is better)
   R²:  0.996050 → 0.996060 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


### Robust Scaler

In [106]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [107]:
X_train

array([[ 0.6       , -0.16666667,  0.        , ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        ,  0.33333333,  1.        , ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -1.        ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       , -0.16666667,  0.        , ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        ,  0.75      ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.33333333,  0.        , ...,  0.61755563,
         0.59917355,  7.5       ]])

In [108]:
X_test

array([[ 4.00000000e-01,  8.33333333e-01,  0.00000000e+00, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.50000000e-01,  1.00000000e+00, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -5.83333333e-01,  0.00000000e+00, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01,  4.16666667e-01,  1.00000000e+00, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01,  6.66666667e-01,  0.00000000e+00, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  8.33333333e-01,  0.00000000e+00, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]])

#### Default Parameters

In [109]:
training_model = SVR()
training_model

SVR()

In [110]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 1.57 s, sys: 108 ms, total: 1.68 s
Wall time: 1.67 s


SVR()

In [111]:
y_pred = training_model.predict(X_test)
y_pred

array([-0.05075509,  0.30942208,  0.02427418, ...,  0.76234348,
        0.11316258, -0.06404649])

In [112]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [113]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [114]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.007162889519585195
MAE Score: 0.06018914116593269
R2 Score: 0.9941210529212783
RMSE Score: 0.08463385563464065


In [115]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 0.131154 → 0.007163 (lower is better)
   R²:  0.892355 → 0.994121 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [116]:
grid_param_robust = [grid_param_minmax, grid_param_standard]

In [117]:
grid_search = GridSearchCV(
    SVR(), 
    grid_param_robust, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [118]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 2 candidates, totalling 10 fits
CPU times: user 7min 9s, sys: 1.11 s, total: 7min 10s
Wall time: 18min 31s


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid=[{'C': [100], 'coef0': [1.0], 'epsilon': [0.1],
                          'gamma': [0.1], 'kernel': ['rbf'],
                          'shrinking': [True], 'tol': [0.001]},
                         {'C': [100], 'coef0': [1.0], 'epsilon': [0.1],
                          'gamma': [0.001], 'kernel': ['linear'],
                          'shrinking': [True], 'tol': [0.001]}],
             scoring='neg_mean_squared_error', verbose=1)

In [119]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'C': 100, 'coef0': 1.0, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'linear', 'shrinking': True, 'tol': 0.001}
Best Cross-Validation Score (Negative MSE): -0.004711878953281346
Best Cross-Validation Score (MSE): 0.004711878953281346
Best Cross-Validation Score (RMSE): 0.06864312750218587


In [120]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [121]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [122]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004732905750349174
MAE Score: 0.05094341449748287
R2 Score: 0.9961154639676068
RMSE Score: 0.06879611726216221


In [123]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 0.005476 → 0.004733 (lower is better)
   R²:  0.995505 → 0.996115 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


# All Performance

In [124]:
show_model_history('regression', model_name=model_name_for_saving, target_column=target_name_for_saving)


📊 REGRESSION Model Performance History
    Model         Scaler      Target  Features      MSE      MAE     RMSE       R²
SVR Tuned   RobustScaler SolarEnergy        22 0.004733 0.050943 0.068796 0.996115
SVR Tuned   MinMaxScaler SolarEnergy        22 0.004740 0.053711 0.068846 0.996110
SVR Tuned StandardScaler SolarEnergy        22 0.004801 0.051977 0.069290 0.996060
      SVR   MinMaxScaler SolarEnergy        22 0.005304 0.057801 0.072828 0.995647
      SVR StandardScaler SolarEnergy        22 0.005930 0.060641 0.077008 0.995133
      SVR   RobustScaler SolarEnergy        22 0.007163 0.060189 0.084634 0.994121

📈 Total models shown: 6
🏆 Best R² Score: 0.996115 (SVR Tuned + RobustScaler for SolarEnergy)


# <center><font size="50" color="red">Thank You</font></center>